# <span style="color:#0b486b">  FIT3181: Deep Learning (2022)</span>
***
*CE/Lecturer:* Dr **Trung Le** | trunglm@monash.edu <br/>
*Head Tutor:* Mr **Thanh Nguyen** | thanh.nguyen4@monash.edu <br/>
<br/>
Department of Data Science and AI, Faculty of Information Technology, Monash University, Australia
***

# <span style="color:#0b486b">  Student Information</span>
Surname: **Yifan**  <br/>
Firstname: **Dai**    <br/>
Student ID: **31510477**    <br/>
Email: **ydai0019@student.monash.edu**    <br/>
Your tutorial time: **Thu 10am - 12am**    <br/>
***

## <span style="color:#0b486b">Assignment 02: Neural Embedding and Sequence Modelling</span>
### Due: <span style="color:red">11:59pm 23 October 2022</span>  (Sunday)

#### <span style="color:red">Important note:</span> This is an **individual** assignment. It contributes **20%** to you final mark. Read the assignment instruction carefully.

## <span style="color:#0b486b">Instructions</span>

This notebook has been prepared for your to complete Assignment 2. The theme of this assignment is about practical machine learning knowledge and skills in deep neural networks, word embedding and text analytics. Some sections have been partially completed to help you get
started. **The total marks for this notebook is 100**.

* Before you start, read the entire notebook carefully once to understand what you need to do. <br><br>
* For each cell marked with **#YOU ARE REQUIRED TO INSERT YOUR CODES IN THIS CELL**, there will be places where you **must** supply your own codes when instructed. <br>

This assignment contains **five** parts:

* Part 1: Questions on downloading and preprocessing data **[5 points]**
* Part 2: Questions on using Word2Vect to transform texts to vectors **[20 points]**
* Part 3: Coding assessment on Text CNN for sequence modeling and neural embedding **[10 points]**
* Part 4: Coding assessment on BERT for a feature extraction **[10 points]**
* Part 5: Coding assessment on RNNs for sequence modeling and neural embedding **[55 points]**


**Hint**: This assignment was essentially designed based on the lectures and tutorials sessions covered from Weeks 7 to 10. You are strongly recommended to go through these contents thoroughly which might help you to complete this assignment.

## <span style="color:#0b486b">What to submit</span>

This assignment is to be completed individually and submitted to Moodle unit site. **By the due date, you are required to submit one  <span style="color:red; font-weight:bold">single zip file, named xxx_assignment02_solution.zip</span> where `xxx` is your student ID, to the corresponding Assignment (Dropbox) in Moodle**. 

***For example, if your student ID is <span style="color:red; font-weight:bold">12356</span>, then gather all of your assignment solution to folder, create a zip file named <span style="color:red; font-weight:bold">123456_assignment02_solution.zip</span> and submit this file.***

Within this zip folder, you **must** submit the following files:
1.	**Assignment02_solution.ipynb**:  this is your Python notebook solution source file.
1.	**Assignment02_output.html**: this is the output of your Python notebook solution *exported* in html format.
1.	Any **extra files or folder** needed to complete your assignment (e.g., images used in your answers).

## <span style="color:#0b486b">Set random seeds</span>

We start with importing tensorflow and numpy and setting random seeds for TF and numpy. You can use any seeds you prefer.

In [25]:
import numpy as np
import tensorflow as tf

tf.random.set_seed(6789)
np.random.seed(6789)

## <span style="color:#0b486b">Part 1: Download and preprocess the data</span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 5 points]<span></div>

The dataset we use for this assignment is a question classification dataset for which the train set consists of $5,500$ questions belonging to 6 coarse question categories including:
- abbreviation (ABBR), 
- entity (ENTY), 
- description (DESC), 
- human (HUM), 
- location (LOC) and 
- numeric (NUM).


Preprocessing data is an inital and important step in any machine learning or deep learning projects. The following *DataManager* class helps you to download data and preprocess data for the later questions of this assignment. 

In [2]:
import os
import zipfile
import collections
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')

class DataManager:
    def __init__(self, verbose=True, maxlen= 50, random_state=6789):
        self.verbose = verbose
        self.max_sentence_len = 0
        self.str_questions = list()
        self.str_labels = list()
        self.numeral_labels = list()
        self.maxlen = maxlen
        self.numeral_data = list()
        self.random_state = random_state
        self.random = np.random.RandomState(random_state)
        
    @staticmethod
    def maybe_download(dir_name, file_name, url, verbose= True):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        if not os.path.exists(os.path.join(dir_name, file_name)):
            urlretrieve(url + file_name, os.path.join(dir_name, file_name))
        if verbose:
            print("Downloaded successfully {}".format(file_name))
    
    def read_data(self, dir_name, file_names):
        self.str_questions= list(); self.str_labels= list()
        for file_name in file_names:
            file_path= os.path.join(dir_name, file_name)
            with open(file_path, "r", encoding="latin-1") as f:
                for row in f:
                    row_str= row.split(":")
                    label, question= row_str[0], row_str[1]
                    question= question.lower()
                    self.str_labels.append(label)
                    self.str_questions.append(question[0:-1])
                    if self.max_sentence_len < len(self.str_questions[-1]):
                        self.max_sentence_len= len(self.str_questions[-1])
         
        # turns labels into numbers
        le= preprocessing.LabelEncoder()
        le.fit(self.str_labels)
        self.numeral_labels = np.array(le.transform(self.str_labels))
        self.str_classes= le.classes_
        self.num_classes= len(self.str_classes)
        if self.verbose:
            print("\nSample questions... \n")
            print(self.str_questions[0:5])
            print("Labels {}\n\n".format(self.str_classes))
    
    def manipulate_data(self):
        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.fit_on_texts(self.str_questions)
        self.numeral_data = tokenizer.texts_to_sequences(self.str_questions)
        self.numeral_data = tf.keras.preprocessing.sequence.pad_sequences(self.numeral_data, padding='post', truncating= 'post', maxlen= self.maxlen)
        self.word2idx = tokenizer.word_index
        self.word2idx = {k:v for k,v in self.word2idx.items()}
        self.idx2word = {v:k for k,v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
    
    def train_valid_split(self, train_ratio=0.9):
        idxs = np.random.permutation(np.arange(len(self.str_questions)))
        train_size = int(train_ratio*len(idxs)) +1
        self.train_str_questions, self.valid_str_questions = self.str_questions[0:train_size], self.str_questions[train_size:]
        self.train_numeral_data, self.valid_numeral_data = self.numeral_data[0:train_size], self.numeral_data[train_size:]
        self.train_numeral_labels, self.valid_numeral_labels = self.numeral_labels[0:train_size], self.numeral_labels[train_size:]
        self.tf_train_set = tf.data.Dataset.from_tensor_slices((self.train_numeral_data, self.train_numeral_labels))
        self.tf_valid_set = tf.data.Dataset.from_tensor_slices((self.valid_numeral_data, self.valid_numeral_labels))

In [3]:
print('Loading data...')
DataManager.maybe_download("Data", "train_1000.label", "http://cogcomp.org/Data/QA/QC/")
DataManager.maybe_download("Data", "TREC_10.label", "http://cogcomp.org/Data/QA/QC/")

dm = DataManager(maxlen=100)
dm.read_data("Data/", ["train_1000.label", "TREC_10.label"])   # read data
# If you want to play around with a bigger dataset, you can try train_set.label

Loading data...
Downloaded successfully train_1000.label
Downloaded successfully TREC_10.label

Sample questions... 

['manner how did serfdom develop in and then leave russia ?', 'cremat what films featured the character popeye doyle ?', "manner how can i find a list of celebrities ' real names ?", 'animal what fowl grabs the spotlight after the chinese year of the monkey ?', 'exp what is the full form of .com ?']
Labels ['ABBR' 'DESC' 'ENTY' 'HUM' 'LOC' 'NUM']




In [4]:
dm.manipulate_data()
dm.train_valid_split(train_ratio=0.8)

You now have a data manager, named *dm* containing the training and validiation sets in both text and numeric forms. Your task is to play around and read this code to figure out the meanings of some important attributes that will be used in the next parts.

#### <span style="color:red">**Question 1.1**</span> 
**What is the purpose of `self.train_str_questions` and `self.train_numeral_labels`? Write your code to print out the first five questions with labels in the training set.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

#Your answer here

`self.train_str_questions`is to store questions of training set in str form.  
`self.train_numeral_labels` is a list which store the labels of the training data in numeric form.

In [5]:
#Your code here
for i in range(5):
    print(dm.train_str_questions[i], dm.train_numeral_labels[i])
    print("\n")

manner how did serfdom develop in and then leave russia ? 1


cremat what films featured the character popeye doyle ? 2


manner how can i find a list of celebrities ' real names ? 1


animal what fowl grabs the spotlight after the chinese year of the monkey ? 2


exp what is the full form of .com ? 0




#### <span style="color:red">**Question 1.2**</span> 
**What is the purpose of `self.train_numeral_data`? Write your code to print out the first five questions in the numeric format with labels in the training set.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

#Your answer here

`self.train_numeral_data` is to store tokenized questions of training set in numeric form.



In [6]:
#Your code here
for i in range(5):
    
    print(dm.train_numeral_data[i], dm.train_numeral_labels[i])

[  35   11   18 1030 1031    5   25  561 1032  562    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0] 1
[  43    2  563 1033    1  164 1034 1035    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   

#### <span style="color:red">**Question 1.3**</span> 
**What is the purpose of two dictionaries: `self.word2idx` and `self.idx2word`? Write your code to print out the first five key-value pairs of those dictionaries.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

#Your answer here

`self.word2idx` is the sequence number corresponding to the word in this question

`self.idx2word` is the word in the question corresponding to the serial number



In [7]:
#Your code here
for i,(k,v) in enumerate(dm.idx2word.items()):
    if i in range(0,5):
        print({k:v},end=""+"\n")

{1: 'the'}
{2: 'what'}
{3: 'is'}
{4: 'of'}
{5: 'in'}


#### <span style="color:red">**Question 1.4**</span> 
**What is the purpose of `self.tf_train_set`? Write your code to print out the first five items of `self.tf_train_set`.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

#Your answer here


`self.tf_train_set` is to convert the train_numeral_data and train_numeral_labels to a training Dataset.



In [8]:
#Your code here
i = 0
for tf in dm.tf_train_set:
    i += 1
    print(tf)
    if i == 5:
        break

(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  35,   11,   18, 1030, 1031,    5,   25,  561, 1032,  562,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  43,    2,  563, 1033,    1,  164, 1034, 1035,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,

#### <span style="color:red">**Question 1.5**</span> 
**What is the purpose of `self.tf_valid_set`? Write your code to print out the first five items of `self.tf_valid_set`.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

#Your answer here

`self.tf_valid_set` is very similar to selftftrainset,whichis getting from the `tf.data.Dataset.from.tensor_slices((self.valid numerak_ data,self.valid_numeral_labels))` which is the combination of validation data and corresponding label where `set.valid numeral_data` is a(1201,100matrix and each row is a (100,1)vector.Because the validation data are all numerical data,we can directly use it in our model.Therefore,the purpose of `self.tf_valid_set` is to create a validation set which can be directly used to validate deep learning model performance.


In [9]:
#Your code here

i = 0
for tf in dm.tf_valid_set:
    i += 1
    print(tf)
    if i == 5:
        break


(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  23,    2,   23,   13,  243,    3,   53,    1, 3031,   20,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
(<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([   6,   12,   98,    1, 3032,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,

## <span style="color:#0b486b">Part 2: Using Word2Vect to transform texts to vectors </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

In this part, you will be assessed on how to use a pretrained Word2Vect model for realizing a machine learning task. Basically, you will use this pretrained Word2Vect to transform the questions in the above dataset stored in the *data manager object dm* to numeric form for training a Support Vector Machine in sckit-learn.  

In [24]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#### <span style="color:red">**Question 2.1**</span> 
**Write code to download the pretrained model *glove-wiki-gigaword-100*. Note that this model transforms a word in its dictionary to a $100$ dimensional vector.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [11]:
word2vect = api.load('glove-wiki-gigaword-100')

#### <span style="color:red">**Question 2.2**</span> 

**Write code for the function *get_word_vector(word, model)* used to transform a word to a vector using the pretrained Word2Vect model *model*. Note that for a word not in the vocabulary of our *word2vect*, you need to return a vector $0$ with 100 dimensions.**

<div style="text-align: right"><span style="color:red">[3 points]</span></div> 

In [12]:
def get_word_vector(word, model):
    try:
        vector = model.fet_vector(word)
    except: #word not in the vocabulary
        vector = np.zeros(100)
    return vector

#### <span style="color:red">**Question 2.3**</span> 

**Write the code for the function `get_sentence_vector(sentence, important_score=None, model= None)`. Note that this function will transform a sentence to a 100-dimensional vector using the pretrained model *model*. In addition, the list *important_score* which has the same length as the *sentence* specifies the important scores of the words in the sentence. In your code, you first need to apply *softmax* function over *important_score* to obtain the important weight *important_weight* which forms a probability over the words of the sentence. Furthermore, the final vector of the sentence will be weighted sum of the individual vectors for words and the weights in *important_weight*.**
- $important\_weight = softmax(important\_score)$.
- $final\_vector= important\_weight[1]\times v[1] + important\_weight[2]\times v[2] + ...+ important\_weight[L]\times v[L]$ where $L$ is the length of the sentence and $v[i]$ is the vector representation of the $i-th$  word in this sentence.

**Note that if `important_score=None` is set by default, your function should return the average of all representation vectors corresponding to set `important_score=[1,1,...,1]`.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div> 

In [13]:
def softmax(x):
    
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [14]:
def get_sentence_vector(sentence, important_score=None, model= None):
    mysentence = sentence.split(' ')
    L = len(mysentence)
    final_vector= np.zeros(100)
    if important_score is None:
        important_score = np.ones(L)
    else:
        important_weight = softmax(important_score)
    
    for i in range (L):
        vector_i = get_word_vector(mysentence[i], model)
        final_vector = final_vector + important_score[i]*vector_i
    
    return final_vector

#### <span style="color:red">**Question 2.4**</span> 

**Write code to transform the training questions in *dm.train_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_train$ which is an numpy array of the feature vectors and $y\_train$ which is an array of numeric labels (*Hint: dm.train_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [15]:
print("Transform training set to feature vectors...")
my_important_score = np.logspace(0,100,100,base=0.9)
final_vector=list()
for i in dm.train_str_questions:
    final_vector.append( get_sentence_vector(i, important_score = None, model= word2vect) )
X_train = np.array(final_vector,dtype="float32")
y_train = dm.train_numeral_labels.reshape(-1,1)

Transform training set to feature vectors...


#### <span style="color:red">**Question 2.5**</span> 

**Write code to transform the training questions in *dm.valid_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_valid$ which is an numpy array of the feature vectors and $y\_valid$ which is an array of numeric labels (*Hint: dm.valid_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [16]:
print("Transform valid set to feature vectors...")
important_score = np.array([0.9**i for i in range(100)])
X_valid= [get_sentence_vector(line, important_score, word2vect) for line in dm.valid_str_questions]
y_valid= dm.valid_numeral_labels

Transform valid set to feature vectors...


#### <span style="color:red">**Question 2.6**</span> 

**It is now to use *MinMaxScaler(feature_range=(-1,1))* in sckit-learn to scale both training and valid sets to the range $(-1,1)$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [17]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
scaler.fit(X_valid)
X_valid = scaler.fit_transform(X_valid)

#### <span style="color:red">**Question 2.7**</span> 

**Declare a support vector machine (the class *SVC*  in sckit-learn) with RBF kernel, $C=1$, $gamma= 2^{-3}$ and fit on the training set.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [18]:
svm = SVC(C=1.0, kernel='rbf',gamma=1/pow(2,3))
svm.fit(X_train,y_train)

C:\Users\HP\anaconda3\envs\tf2_cpu\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(gamma=0.125)

#### <span style="color:red">**Question 2.8**</span> 

**Finally, we use the trained *svm* to evaluate on the valid set $X\_valid$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [19]:
y_valid_pred = svm.predict(X_valid)
acc = sum(y_valid_pred == y_valid)/len(y_valid_pred)
print(acc)

0.2140468227424749


## <span style="color:#0b486b">Part 3: Text CNN for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 3.1**</span> 

**In what follows, you are required to complete the code for Text CNN for sentence classification. The paper of Text CNN can be found at this [link](https://www.aclweb.org/anthology/D14-1181.pdf). Here is the description of the Text CNN that you need to construct.**
- There are three attributes (properties or instance variables): *embed_size, state_size, data_manager*.
  - `embed_size`: the dimension of the vector space for which the words are embedded to using the embedding matrix.
  - `state_size`: the number of filters used in *Conv1D* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)).
  - `data_manager`: the data manager to store information of the dataset.
- The detail of the computational process is as follows:
  - Given input $x$, we embed $x$ using the embedding matrix to obtain an $3D$ tensor $[batch\_size \times vocab\_size \times embed\_size]$ as $h$.
  - We feed $h$ to three Convd 1D layers, each of which has $state\_size$ filters, padding=same, activation= relu, and $kernel\_size= 3, 5, 7$ respectively to obtain $h1, h2, h3$. Note that each $h1, h2, h3$ is a 3D tensor with the shape $[batch\_size \times output\_size \times state\_size]$.
  - We then apply *GlobalMaxPool1D()* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool1D)) over $h1, h2, h3$ to obtain 2D tensors stored in $h1, h2, h3$ again.
  - We then concatenate three 2D tensors $h1, h2, h3$ to obtain $h$. Note that you need to specify the axis to concatenate.
  - We finally build up one dense layer on the top of $h$ for classification.
  
  <div style="text-align: right"><span style="color:red">[8 points]</span></div>
  

In [22]:
class TextCNN:
    def __init__(self, embed_size= 128, state_size=16, data_manager=None):
        self.data_manager = data_manager
        self.embed_size = embed_size
        self.state_size = state_size
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.data_manager.vocab_size +1, self.embed_size)(x)
        h1 = tf.keras.layers.Conv1D(self.state_size, 3, padding='same', activation='relu')(h) 
        h2 = tf.keras.layers.Conv1D(self.state_size, 5, padding='same', activation='relu')(h)
        h3 = tf.keras.layers.Conv1D(self.state_size, 7, padding='same', activation='relu')(h)
        h1 = tf.keras.layers.GlobalMaxPool1D()(h1)
        h2 = tf.keras.layers.GlobalMaxPool1D()(h2)
        h3 = tf.keras.layers.GlobalMaxPool1D()(h3)
        h = tf.keras.layers.Concatenate(axis=1)([h1, h2, h3])
        h = tf.keras.layers.Dense(self.data_manager.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h) 
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)


#### <span style="color:red">**Question 3.2**</span> 
**Here is the code to test TextCNN above. You can observe that TextCNN outperforms the traditional approach SVM + Word2Vect for this task. The reason is that TextCNN enables us to automatically learn the feature that fits to the task. This makes deep learning different from hand-crafted feature approaches. Complete the code to test the model. Note that when compiling the model, you can use the Adam optimizer.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [26]:
text_cnn = TextCNN(data_manager=dm)
text_cnn.build()
text_cnn.compile_model(optimizer="adam",loss='sparse_categorical_crossentropy',metrics=["accuracy"])
text_cnn.model.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 1s 22ms/step - loss: 1.7150 - accuracy: 0.4621 - val_loss: 1.6090 - val_accuracy: 0.7023
Epoch 2/20
19/19 [==============================] - 0s 10ms/step - loss: 1.4420 - accuracy: 0.8893 - val_loss: 1.3043 - val_accuracy: 0.7860
Epoch 3/20
19/19 [==============================] - 0s 12ms/step - loss: 1.0362 - accuracy: 0.9026 - val_loss: 0.8614 - val_accuracy: 0.8395
Epoch 4/20
19/19 [==============================] - 0s 11ms/step - loss: 0.5884 - accuracy: 0.9292 - val_loss: 0.5271 - val_accuracy: 0.8829
Epoch 5/20
19/19 [==============================] - 0s 11ms/step - loss: 0.3006 - accuracy: 0.9625 - val_loss: 0.3500 - val_accuracy: 0.9064
Epoch 6/20
19/19 [==============================] - 0s 11ms/step - loss: 0.1724 - accuracy: 0.9717 - val_loss: 0.2536 - val_accuracy: 0.9197
Epoch 7/20
19/19 [==============================] - 0s 10ms/step - loss: 0.1104 - accuracy: 0.9808 - val_loss: 0.1950 - val_accuracy: 0.9431
Epoch 8/20
19

## <span style="color:#0b486b">Part 4: Sentence representation with BERT </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 4.1**</span>
**Use a pretrained BERT model to extract feaure vectors for the training and valid sets. You should choose an appropriate pretrained BERT model that fits your computational resource.**
<div style="text-align: right"><span style="color:red">[6 points]</span></div>

In [27]:
#Insert your code for extracting training set


#### <span style="color:red">**Question 4.2**</span>
**Use Logistic Regression to train on the training set and then evaluate on the valid set.**
<div style="text-align: right"><span style="color:red">[4 points]</span></div>

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

C:\Users\HP\anaconda3\envs\tf2_cpu\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
y_valid_pred = clf.predict(X_valid)
acc = accuracy_score(y_valid,y_valid_pred)
print(acc)

0.2140468227424749


## <span style="color:#0b486b">Part 5: RNNs for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 55 points]<span></div>

### <span style="color:#0b486b">5.1. One-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 5.1.1**</span> 
**In this part, you need to construct an RNN to learn from the dataset of interest. Basically, you are required first to construct the class UniRNN (Uni-directional RNN) with the following requirements:**
- Attribute `data_manager (self.data_manager)`: specifies the data manager used to store data for the model.
- Attribute `cell_type (self.cell_type)`: can receive three values including `basic_rnn`, `gru`, and `lstm` which specifies the memory cells formed a hidden layer.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes  of memory cells. For example, $state\_sizes = [64, 128]$ means that you have two hidden layers in your network with hidden sizes of $64$ and $128$ respectively.

**Note that when declaring an embedding layer for the network, you need to set *mask_zero=True* so that the padding zeros in the sentences will be masked and ignored. This helps to have variable length RNNs. For more detail, you can refer to this [link](https://www.tensorflow.org/guide/keras/masking_and_padding).**

<div style="text-align: right"><span style="color:red">[7 points]</span></div>

In [31]:
class UniRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1 
        
    #return the correspoding memory cell
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return tf.keras.layers.GRU(state_size, return_sequences= return_sequences, activation = activation)
        elif cell_type== 'lstm':
            return tf.keras.layers.LSTM(state_size, return_sequences= return_sequences, activation = activation)
        else:
            return tf.keras.layers.SimpleRNN(state_size, return_sequences= return_sequences, activation = activation)
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.vocab_size, self.embed_size)(x)
        num_layers = len(self.state_sizes)
        for i in range(num_layers):
            h =  self.get_layer(cell_type = self.cell_type, state_size = self.state_sizes[i], return_sequences= (i != (num_layers-1)))(h)
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
   
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 5.1.2**</span> 
**Run with basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [32]:
uni_rnn = UniRNN(cell_type= 'basic_rnn', embed_size= 128, state_sizes= [64, 128], data_manager = dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 3s 69ms/step - loss: 1.5819 - accuracy: 0.2948 - val_loss: 1.4160 - val_accuracy: 0.3344
Epoch 2/20
19/19 [==============================] - 1s 51ms/step - loss: 1.3846 - accuracy: 0.3972 - val_loss: 1.3816 - val_accuracy: 0.3177
Epoch 3/20
19/19 [==============================] - 1s 47ms/step - loss: 1.4084 - accuracy: 0.4005 - val_loss: 1.3661 - val_accuracy: 0.2843
Epoch 4/20
19/19 [==============================] - 1s 47ms/step - loss: 1.3421 - accuracy: 0.3722 - val_loss: 1.3927 - val_accuracy: 0.2843
Epoch 5/20
19/19 [==============================] - 1s 46ms/step - loss: 1.2926 - accuracy: 0.4355 - val_loss: 2.2285 - val_accuracy: 0.0067
Epoch 6/20
19/19 [==============================] - 1s 45ms/step - loss: 1.3748 - accuracy: 0.4438 - val_loss: 2.0144 - val_accuracy: 0.0234
Epoch 7/20
19/19 [==============================] - 1s 46ms/step - loss: 1.3849 - accuracy: 0.3822 - val_loss: 1.1176 - val_accuracy: 0.6789
Epoch 8/20
19

#### <span style="color:red">**Question 5.1.3**</span> 
**Run with GRU ('gru') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [33]:
uni_rnn = UniRNN(cell_type= 'gru', embed_size= 128, state_sizes= [64, 128], data_manager = dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 6s 140ms/step - loss: 1.6870 - accuracy: 0.2182 - val_loss: 1.6834 - val_accuracy: 0.2040
Epoch 2/20
19/19 [==============================] - 2s 104ms/step - loss: 1.6648 - accuracy: 0.2182 - val_loss: 1.6854 - val_accuracy: 0.2040
Epoch 3/20
19/19 [==============================] - 2s 106ms/step - loss: 1.6638 - accuracy: 0.2157 - val_loss: 1.6862 - val_accuracy: 0.2040
Epoch 4/20
19/19 [==============================] - 2s 101ms/step - loss: 1.6632 - accuracy: 0.2157 - val_loss: 1.6868 - val_accuracy: 0.2040
Epoch 5/20
19/19 [==============================] - 2s 102ms/step - loss: 1.6628 - accuracy: 0.2157 - val_loss: 1.6872 - val_accuracy: 0.2040
Epoch 6/20
19/19 [==============================] - 2s 104ms/step - loss: 1.6625 - accuracy: 0.2157 - val_loss: 1.6878 - val_accuracy: 0.2040
Epoch 7/20
19/19 [==============================] - 2s 103ms/step - loss: 1.6621 - accuracy: 0.2173 - val_loss: 1.6883 - val_accuracy: 0.2977
Epoch 

#### <span style="color:red">**Question 5.1.4**</span> 
**Run with LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [34]:
uni_rnn = uni_rnn = UniRNN(cell_type= 'lstm', embed_size= 128, state_sizes= [64, 128], data_manager = dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 6s 145ms/step - loss: 1.6943 - accuracy: 0.2298 - val_loss: 1.6901 - val_accuracy: 0.2040
Epoch 2/20
19/19 [==============================] - 2s 103ms/step - loss: 1.6670 - accuracy: 0.2190 - val_loss: 1.6882 - val_accuracy: 0.2040
Epoch 3/20
19/19 [==============================] - 2s 104ms/step - loss: 1.6653 - accuracy: 0.2206 - val_loss: 1.6874 - val_accuracy: 0.2040
Epoch 4/20
19/19 [==============================] - 2s 105ms/step - loss: 1.6641 - accuracy: 0.2157 - val_loss: 1.6870 - val_accuracy: 0.2040
Epoch 5/20
19/19 [==============================] - 2s 106ms/step - loss: 1.6632 - accuracy: 0.2157 - val_loss: 1.6867 - val_accuracy: 0.2040
Epoch 6/20
19/19 [==============================] - 2s 104ms/step - loss: 1.6625 - accuracy: 0.2157 - val_loss: 1.6867 - val_accuracy: 0.2977
Epoch 7/20
19/19 [==============================] - 2s 108ms/step - loss: 1.6619 - accuracy: 0.2306 - val_loss: 1.6867 - val_accuracy: 0.2977
Epoch 

### <span style="color:#0b486b">5.2. Bi-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 5.2.1**</span> 
**In what follow, you will investigate BiRNN. The task is similar to Part 5.1 but you need to write the code for an BiRNN. Note that the function *get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh')* has to return the hidden layer with bidirectional memory cells (e.g., Basic RNN, GRU, and LSTM cells).**

**Complete the code of the class *BiRNN*. Note that for the embedding layer you need to set *mask_zero=True*.**

<div style="text-align: right"><span style="color:red">[7 points]</span></div>

In [35]:
class BiRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1
        
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return tf.keras.layers.Bidirectional(
                tf.keras.layers.GRU(state_size, return_sequences= return_sequences, activation = activation)) 
        elif cell_type== 'lstm':
            return tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(state_size, return_sequences= return_sequences, activation = activation))
        else:
            return tf.keras.layers.Bidirectional(
                tf.keras.layers.SimpleRNN(state_size, return_sequences= return_sequences, activation = activation))
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.vocab_size, self.embed_size)(x)
        num_layers  = len(self.state_sizes) 
        for i in range(num_layers):
            h = h =  self.get_layer(cell_type = self.cell_type, state_size = self.state_sizes[i], return_sequences= (i != (num_layers-1)))(h)#Insert your code here, add more lines if necessary
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)      


#### <span style="color:red">**Question 5.2.2**</span> 
**Run BiRNN for basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [36]:
bi_rnn = bi_rnn = BiRNN(cell_type= 'basic_rnn', embed_size= 128, state_sizes= [64, 128], data_manager= dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 5s 119ms/step - loss: 1.1304 - accuracy: 0.5953 - val_loss: 0.5282 - val_accuracy: 0.8395
Epoch 2/20
19/19 [==============================] - 2s 96ms/step - loss: 0.2133 - accuracy: 0.9459 - val_loss: 0.2959 - val_accuracy: 0.9064
Epoch 3/20
19/19 [==============================] - 2s 91ms/step - loss: 0.0703 - accuracy: 0.9883 - val_loss: 0.2539 - val_accuracy: 0.9231
Epoch 4/20
19/19 [==============================] - 2s 90ms/step - loss: 0.0198 - accuracy: 0.9992 - val_loss: 0.1703 - val_accuracy: 0.9398
Epoch 5/20
19/19 [==============================] - 2s 91ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.1639 - val_accuracy: 0.9498
Epoch 6/20
19/19 [==============================] - 2s 91ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1689 - val_accuracy: 0.9532
Epoch 7/20
19/19 [==============================] - 2s 91ms/step - loss: 6.5404e-04 - accuracy: 1.0000 - val_loss: 0.1763 - val_accuracy: 0.9532
Epoch 8/

#### <span style="color:red">**Question 5.2.3**</span> 
**Run BiRNN for GRU ('gru') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [37]:
bi_rnn = BiRNN(cell_type= 'gru', embed_size= 128, state_sizes= [64, 128], data_manager= dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 12s 290ms/step - loss: 1.3976 - accuracy: 0.4738 - val_loss: 0.9426 - val_accuracy: 0.8428
Epoch 2/20
19/19 [==============================] - 4s 207ms/step - loss: 0.4659 - accuracy: 0.8893 - val_loss: 0.5082 - val_accuracy: 0.8796
Epoch 3/20
19/19 [==============================] - 4s 203ms/step - loss: 0.1493 - accuracy: 0.9600 - val_loss: 0.4435 - val_accuracy: 0.8027
Epoch 4/20
19/19 [==============================] - 4s 202ms/step - loss: 0.0676 - accuracy: 0.9825 - val_loss: 0.2248 - val_accuracy: 0.9264
Epoch 5/20
19/19 [==============================] - 4s 203ms/step - loss: 0.0182 - accuracy: 0.9975 - val_loss: 0.2002 - val_accuracy: 0.9431
Epoch 6/20
19/19 [==============================] - 4s 206ms/step - loss: 0.0390 - accuracy: 0.9908 - val_loss: 0.1493 - val_accuracy: 0.9498
Epoch 7/20
19/19 [==============================] - 4s 205ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.1536 - val_accuracy: 0.9498
Epoch

##### <span style="color:red">**Question 5.2.4**</span> 
**Run BiRNN for LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [38]:
bi_rnn = BiRNN(cell_type= 'lstm', embed_size= 128, state_sizes= [64, 128], data_manager= dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 13s 286ms/step - loss: 1.6278 - accuracy: 0.2798 - val_loss: 1.4057 - val_accuracy: 0.3144
Epoch 2/20
19/19 [==============================] - 4s 233ms/step - loss: 0.9465 - accuracy: 0.6811 - val_loss: 1.8096 - val_accuracy: 0.3913
Epoch 3/20
19/19 [==============================] - 4s 220ms/step - loss: 0.4672 - accuracy: 0.8859 - val_loss: 0.4390 - val_accuracy: 0.8963
Epoch 4/20
19/19 [==============================] - 4s 232ms/step - loss: 0.2392 - accuracy: 0.9359 - val_loss: 0.4548 - val_accuracy: 0.8796
Epoch 5/20
19/19 [==============================] - 4s 222ms/step - loss: 0.1324 - accuracy: 0.9625 - val_loss: 0.4259 - val_accuracy: 0.8997
Epoch 6/20
19/19 [==============================] - 4s 217ms/step - loss: 0.0606 - accuracy: 0.9825 - val_loss: 0.3045 - val_accuracy: 0.9298
Epoch 7/20
19/19 [==============================] - 4s 218ms/step - loss: 0.1428 - accuracy: 0.9659 - val_loss: 0.2187 - val_accuracy: 0.9365
Epoch

### <span style="color:#0b486b">5.3. RNNs with various types, cells, and fine-tuning embedding matrix for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 11 points]<span></div>

#### <span style="color:red">**Question 5.3.1**</span> 

**In what follows, you are required to combine the code in Part 1 and Part 2 to gain a general RNN which can be either Uni-directional RNN or Bi-directional RNN and the embedding matrix can be initialized using a pretrained Word2Vect.**

**Below are the descriptions of the attributes of the class *RNN*:**
- `run_mode (self.run_mode)` has three values (scratch, init-only, and init-fine-tune).
  - `scratch` means training the embedding matrix from scratch.
  - `init-only` means only initializing the embedding matrix with a pretrained Word2Vect but **not further doing** fine-tuning that matrix.
  - `init-fine-tune` means both initializing the embedding matrix with a pretrained Word2Vect and **further doing** fine-tuning that matrix.
- `network_type (self.network_type)` has two values (uni-directional and bi-directional) which correspond to either Uni-directional RNN or Bi-directional RNN.
- `cell_type (self.cell_type)` has three values (simple-rnn, gru, and lstm) which specify the memory cell used in the network.
- `embed_model (self.embed_model)` specifes the pretrained Word2Vect model used.
-  `embed_size (self.embed_size)` specifes the embedding size. Note that when run_mode is either init-only' or 'init-fine-tune', this embedding size is extracted from embed_model for dimension compatability.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes  of memory cells. For example, $state\_sizes = [64, 128]$ means that you have two hidden layers in your network with hidden sizes of $64$ and $128$ respectively.

**Complete the code of the class *RNN*.**

<div style="text-align: right"><span style="color:red">[7 points]</span></div>

In [39]:
class RNN:
    def __init__(self, run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = None):
        self.run_mode = run_mode
        self.data_manager = data_manager
        self.cell_type = cell_type
        self.network_type = network_type
        self.state_sizes = state_sizes
        self.embed_model = embed_model
        self.embed_size = embed_size
        if self.run_mode != 'scratch':
            self.embed_size = int(self.embed_model.split("-")[-1])
        self.data_manager = data_manager
        self.vocab_size = dm.vocab_size +1
        self.word2idx = dm.word2idx
        self.word2vect = None
        self.embed_matrix = np.zeros(shape= [self.vocab_size, self.embed_size])
    
    def build_embedding_matrix(self):
        self.word2vect = api.load(self.embed_model)   
        for word, idx in self.word2idx.items():
            try:
                self.embed_matrix[idx] = self.word2vect.word_vec(word)  
                    
            except KeyError: 
                pass            
    
    
    @staticmethod
    def get_layer(cell_type= 'gru', network_type= 'uni-directional', hidden_size= 128, return_sequences= False, activation = 'tanh'):
        if network_type== 'uni-directional':
            if cell_type=='gru':
                return tf.keras.layers.GRU(hidden_size,return_sequences= return_sequences, activation = activation)
            elif cell_type== 'lstm':
                return tf.keras.layers.LSTM(hidden_size,return_sequences= return_sequences, activation = activation)
            else:
                return tf.keras.layers.SimpleRNN(hidden_size,return_sequences= return_sequences, activation = activation)
    
        if network_type== 'bi-directional':
            if cell_type=='gru':
                return tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_size,return_sequences= return_sequences,
                                                                         activation = activation))
            elif cell_type== 'lstm':
                return tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_size,return_sequences= return_sequences, 
                                                                          activation = activation))
            else:
                return tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(hidden_size,return_sequences= return_sequences, 
                                                                               activation = activation))
    
    
    def build(self):
        #Insert your code here
        x = tf.keras.layers.Input(shape=[None])
        if self.run_mode == "scratch":
            self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, trainable= True,mask_zero=True)
        elif self.run_mode == "init-only":
            self.build_embedding_matrix()
            self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, 
                                                        weights= [self.embed_matrix], trainable= False,mask_zero=True) 
        else : #fine-tuned
            self.build_embedding_matrix()
            self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, 
                                                        weights= [self.embed_matrix], trainable= True,mask_zero=True) 
        h = self.embedding_layer(x)
        num_layers = len(self.state_sizes)
        for i in range(num_layers):
            if i != num_layers-1:
                layer = self.get_layer(hidden_size=self.state_sizes[i],return_sequences= True)
                h=layer(h)
            else:
                layer = self.get_layer(hidden_size=self.state_sizes[i])
                h=layer(h)                                                 
           
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
        
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)   

#### <span style="color:red">**Question 5.3.2**</span> 

**Design the experiment to compare three running modes. Note that you should stick with fixed values for other attributes and only vary *run_mode*. Give your comments for the results.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [40]:

for mode in ['scratch','init-fine-tune','init-only']:
    
    
    if mode == 'init-fine-tune':       
        mode_name = 'init-fine-tune'
    elif mode == 'init-only':
        mode_name = 'init-only'
    else:
        mode_name = mode 
    print(mode)
    run_model = RNN(run_mode = mode,data_manager = dm,network_type = 'uni-directional' )    
    run_model.build()
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    run_model.compile_model(optimizer = opt,loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
    
    globals()[mode_name+'history']=run_model.fit(dm.tf_train_set.batch(64),epochs=20,validation_data=dm.tf_valid_set.batch(64),verbose=2)
    

scratch
Epoch 1/20
19/19 - 12s - loss: 1.6279 - accuracy: 0.2989 - val_loss: 1.2730 - val_accuracy: 0.6321
Epoch 2/20
19/19 - 2s - loss: 0.9845 - accuracy: 0.6370 - val_loss: 0.7066 - val_accuracy: 0.7525
Epoch 3/20
19/19 - 2s - loss: 0.4942 - accuracy: 0.8684 - val_loss: 0.3913 - val_accuracy: 0.8896
Epoch 4/20
19/19 - 2s - loss: 0.2198 - accuracy: 0.9484 - val_loss: 0.2476 - val_accuracy: 0.9365
Epoch 5/20
19/19 - 2s - loss: 0.0986 - accuracy: 0.9833 - val_loss: 0.2144 - val_accuracy: 0.9431
Epoch 6/20
19/19 - 2s - loss: 0.0490 - accuracy: 0.9917 - val_loss: 0.1963 - val_accuracy: 0.9498
Epoch 7/20
19/19 - 2s - loss: 0.0205 - accuracy: 0.9983 - val_loss: 0.1992 - val_accuracy: 0.9565
Epoch 8/20
19/19 - 2s - loss: 0.0103 - accuracy: 0.9992 - val_loss: 0.2091 - val_accuracy: 0.9532
Epoch 9/20
19/19 - 2s - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.2242 - val_accuracy: 0.9498
Epoch 10/20
19/19 - 2s - loss: 0.0090 - accuracy: 0.9967 - val_loss: 0.2358 - val_accuracy: 0.9431
Epoch 11/2

C:\Users\HP\AppData\Local\Temp\ipykernel_11136\3359736058.py:23: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  self.embed_matrix[idx] = self.word2vect.word_vec(word)


Epoch 1/20
19/19 - 12s - loss: 1.5378 - accuracy: 0.3930 - val_loss: 1.1161 - val_accuracy: 0.7191
Epoch 2/20
19/19 - 2s - loss: 1.0239 - accuracy: 0.6478 - val_loss: 0.6863 - val_accuracy: 0.8060
Epoch 3/20
19/19 - 2s - loss: 0.5827 - accuracy: 0.8243 - val_loss: 0.4257 - val_accuracy: 0.8729
Epoch 4/20
19/19 - 2s - loss: 0.2730 - accuracy: 0.9301 - val_loss: 0.2858 - val_accuracy: 0.8997
Epoch 5/20
19/19 - 2s - loss: 0.1378 - accuracy: 0.9617 - val_loss: 0.2312 - val_accuracy: 0.9130
Epoch 6/20
19/19 - 2s - loss: 0.0815 - accuracy: 0.9800 - val_loss: 0.1780 - val_accuracy: 0.9532
Epoch 7/20
19/19 - 2s - loss: 0.0505 - accuracy: 0.9875 - val_loss: 0.1392 - val_accuracy: 0.9498
Epoch 8/20
19/19 - 2s - loss: 0.0350 - accuracy: 0.9892 - val_loss: 0.1361 - val_accuracy: 0.9431
Epoch 9/20
19/19 - 2s - loss: 0.0261 - accuracy: 0.9917 - val_loss: 0.1651 - val_accuracy: 0.9398
Epoch 10/20
19/19 - 2s - loss: 0.0162 - accuracy: 0.9942 - val_loss: 0.1409 - val_accuracy: 0.9398
Epoch 11/20
19/19 

#### <span style="color:red">**Question 5.3.3**</span> 

**Run the above general RNN with at least five parameter sets and try to obtain the best performance. You can stick with the running mode *init-fine-tune* and use grid search to tune other parameters. Record your best model which will be used in the next part.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

#Report your results here

Model 1 (run_mode ='init-fine-tune',...): accuracy = ...

......................................................

In [41]:
my_best_rnn = RNN(run_mode ='init-fine-tune',network_type = 'uni-directional',cell_type= 'lstm', embed_model= 'glove-wiki-gigaword-100',
                  embed_size= 128, state_sizes = [64, 64], data_manager = dm)
my_best_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
my_best_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
my_best_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

C:\Users\HP\AppData\Local\Temp\ipykernel_11136\3359736058.py:23: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  self.embed_matrix[idx] = self.word2vect.word_vec(word)


Epoch 1/20
19/19 [==============================] - 12s 219ms/step - loss: 1.5196 - accuracy: 0.3897 - val_loss: 1.1179 - val_accuracy: 0.6154
Epoch 2/20
19/19 [==============================] - 2s 104ms/step - loss: 1.0357 - accuracy: 0.6436 - val_loss: 0.7236 - val_accuracy: 0.7793
Epoch 3/20
19/19 [==============================] - 2s 103ms/step - loss: 0.6207 - accuracy: 0.8135 - val_loss: 0.4688 - val_accuracy: 0.8763
Epoch 4/20
19/19 [==============================] - 2s 108ms/step - loss: 0.3236 - accuracy: 0.9176 - val_loss: 0.2976 - val_accuracy: 0.9030
Epoch 5/20
19/19 [==============================] - 2s 105ms/step - loss: 0.1774 - accuracy: 0.9500 - val_loss: 0.2376 - val_accuracy: 0.9130
Epoch 6/20
19/19 [==============================] - 2s 104ms/step - loss: 0.1041 - accuracy: 0.9667 - val_loss: 0.2275 - val_accuracy: 0.9398
Epoch 7/20
19/19 [==============================] - 2s 105ms/step - loss: 0.0690 - accuracy: 0.9784 - val_loss: 0.1702 - val_accuracy: 0.9532
Epoch

### <span style="color:#0b486b">5.4. RNNs with Attention for Text and Sequence Classification</span> ###

**In what follows, you are required to implement a RNN with the attention machenism for text and sequence classification. This attention mechanism is applied at the last hidden layer of our RNN. Specifically, let $h_1, h_2,...,h_{L-1}, h_L$ be the hidden states at the last hidden layer where $L$ is the sequence length. We compute the context vector $c$ as $c=\sum_{i=1}^{L}a_{i}h_{i}$ where $a_1,...,a_L$ are the allignment weights (i.e., $a_i\geq 0$ and $\sum_{i=1}^{L}a_{i}=1$).**

**The allignment weights are computed as follows:**
- $a=[a_{i}]_{i=1}^{L}=softmax([s_{i}]_{i=1}^{L})$ where $s= [s_{i}]_{i=1}^{L}$ consists of the allignment scores.
- The assigment scores $s= [s_{i}]_{i=1}^{L}$ are computed as $s=tanh(hU)V$ where $h=\left[\begin{array}{c}
h_{1}\\
h_{2}\\
...\\
h_{L-1}\\
h_{L}
\end{array}\right]\in\mathbb{R}^{L\times state\_size_{3}}$, $U\in\mathbb{R}^{state\_size_{3}\times output\_length}$, $V\in\mathbb{R}^{output\_length\times1}$, and $output\_length$ is a hyper-parameter. Note that if we consider a mini-batch, the shape of $h$ is $batch\_size \times L \times state\_size_3$ where $state\_size_3$ is the hidden size of the last hidden layer.

After having the context vector $c$, we concatenate with the last hidden state $h_L$. On top of this concatenation, we conduct the output layer with the softmax activation.

<img src="./images/attention.png" align="center" width=1200/>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 14 points]<span></div>

#### <span style="color:red">**Question 5.4.1**</span>

**We declare the  layer `MyAttention` as a class inherited from `tf.keras.layers.Layer` to realize our attention mechanism. You are required to provide the code for this class. Note that in the `def call(self, all_states, last_state)` method, `all_states` is the collection of all hidden states and `last_state` is the last hidden state.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
class MyAttention(tf.keras.layers.Layer):
    def __init__(self, output_length= 50):
        super(MyAttention, self).__init__()
        self.w = tf.keras.layers.Dense(output_length)
        self.v = tf.keras.layers.Dense(1)
    
    #all_states is the collection of all hidden states and last_state is the last hidden state
    def call(self, all_states, last_state):
        attention_score = self.v(tf.nn.tanh(self.w(all_states)))
        attention_weight = tf.nn.softmax(attention_score,axis=1)
        hiidden_vect=tf.concat([context,last_state],axis=1)
        return hidden_vect
        

#### <span style="color:red">**Question 5.4.2**</span> 

**You are required to extend the class `RNN` in Section `5.3.1` to achieve the class `Attention_RNN` in which the attention mechanism mentioned above is applied at the last hidden layer.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [43]:
class Attention_RNN:
    def __init__(self, run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = None):
        self.run_mode = run_mode
        self.data_manager = data_manager
        self.cell_type = cell_type
        self.network_type = network_type
        self.state_sizes = state_sizes
        self.embed_model = embed_model
        self.embed_size = embed_size
        if self.run_mode != 'scratch':
            self.embed_size = int(self.embed_model.split("-")[-1])
        self.data_manager = data_manager
        self.vocab_size = dm.vocab_size +1
        self.word2idx = dm.word2idx
        self.word2vect = None
        self.embed_matrix = np.zeros(shape= [self.vocab_size, self.embed_size])
    
    def build_embedding_matrix(self):
        #Insert your code here
    
    @staticmethod
    def get_layer(cell_type= 'gru', network_type= 'uni-directional', hidden_size= 128, return_sequences= False, activation = 'tanh'):
        #Insert your code here
    
    def build(self):
        #Insert your code here
        
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


IndentationError: expected an indented block (3251183052.py, line 22)

#### <span style="color:red">**Question 5.4.3**</span> 

**Choose a common setting for standard RNN and RNN with attention and conduct experiments to compare them. A setting here means `cell_type`, `network_type`, and list of `state sizes`**.

<div style="text-align: right"><span style="color:red">[4 points]</span></div>

### <span style="color:#0b486b">5.5. Investigating the embedding vectors from the embedding matrix</span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

**As you know, the embedding matrix is a collection of embedding vectors, each is for one word. In this part, you will base on the cosine similarity of the embedding vectors for the words to find the top-k most relevant words for a given word.**

**Good embeddings should have words close in meaning near each other by some similarity metrics. The similarity metric we'll use is the `consine` similarity, which is defined for two vector $\mathbf{u}$ and $\mathbf{v}$ as $\cos(\mathbf{u}, \mathbf{v})=\frac{\mathbf{u} \cdot \mathbf{v}}{\left\Vert{\mathbf{u}}\right\Vert\left\Vert{\mathbf{v}}\right\Vert}$ where $\cdot$ means dot product and $\left\Vert\cdot\right\Vert$ means the $\mathcal{L}_2$ norm.**

In [ ]:
def cosine_similarity(u,v):
    return np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))

#### <span style="color:red">**Question 5.5.1**</span> 

**You are required to write the code for the function `find_most_similar(word= None, k=5, model= None)`. As its name, this function returns the `top-k most relevant word` for a given word based on the cosine similarity of the embedding vectors.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
def find_most_similar(word= None, k=5, model= None):
    try:
        table = model.word2vect.similar_by_word(word,k)
    except: 
        print("Word is not in the dictionary!")
    return table

Here is the example of the above function. As you can observe, the result makes sense which demonstrates that we obtain a good model with the meaningful embedding matrix.

In [ ]:
find_most_similar(word='poland', k=10, model= my_best_rnn)

#### <span style="color:red">**Question 5.5.2**</span> 
​
**You are required to write the code for the function `plot3D_with_labels(word= None, model= None, k= 10)`. As its name, this function visualizes the `top-k most relevant word` for a given word based on the cosine similarity of the embedding vectors by using tSNE to project the embedding vectors to a 3D space.**
​

<img src="./images/3D_plots.png" align="center" width=600/>
<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
tsne = TSNE(perplexity=30, n_components=3, init='pca', n_iter=5000)
def plot3D_with_labels(word= None, model= None, k= 10):
   #Insert yout codeh here

In [ ]:
plot3D_with_labels(word='poland', k=20, model= rnn)

--- 
<div style="text-align: center"> <span style="color:green">GOOD LUCK WITH YOUR ASSIGNMENT 2!</span> </div>
<div style="text-align: center"> <span style="color:black">END OF ASSIGNMENT</span> </div>